In [1]:
import pandas as pd
import numpy as np

# 데이터 불러오기

- Human Labeling된 데이터인 SKT , 현대백화점, 현대모비스, S-Oil, 기업은행, 셀트리온 데이터 셋을 이용하여 각 정확도를 올린다.

- 순서
    1. 데이터 불러오기
    2. 데이터 분리
    3. Title 의 토큰화 및 불용어 제거
    4. Label 의 원핫인코딩
    5. 길이가 다른 title의 정형화
    6. LSTM을 통한 딥러닝
    7. 정확도 확인

In [2]:
df_skt = pd.read_csv('./trainingset/SKT_traindata_482.csv')
df_train_skt = df_skt.iloc[:200]
df_test_skt = df_skt.iloc[200:]

In [79]:
df_test_skt

,id,Title,label
200,0,"정부, 구글 지도 해외 반출 불허 구글이 보완책 수용 안해 (상보)",0
201,1,"미래부, 2차 유료방송 발전방안 공개토론회 개최",0
202,2,"SKT, 전용폰 구입 고객에 T1 롤드컵 우승 기념품 증정",1
203,3,'갤S7'사면 페이백 50만원 준다더니제2의 거성 모바일 사태?,0
204,4,케이블TV 가입자도 내년 1월부터 SKT 요금할인받는다,1
...,...,...,...
476,78,"스웨덴 에릭슨""4차 산업혁명 협력할 韓 스타트업 찾습니다 ,0\n4187,SKT '...",1
477,79,"CJ헬로비전 작년 영업이익 59.4% 감소한 429억원…""M&A 등 불확실성 해소,...",1
478,80,SKT 2019년까지 11조원 투자,1
479,81,통신사 커버리지맵 정확도 SKT 1위 '오차 0%',1


In [3]:
df_mobis = pd.read_csv('./trainingset/현대모비스_traindata_200.csv', encoding = 'CP949')
df_mobis

,Column1,Title,label
0,0,"코스피, 16일만에 돌아선 外人…보합권 내 혼조",-1
1,1,"코스피, 외인·기관 '사자'에 2200선 돌파",1
2,2,"코스피, 이틀 연속 오르며 1940선 마감… 코스닥 4거래일 만에 600선 회복",1
3,3,"코스피, 외인·기관 '팔자'에 하락마감…2550선은 방어",0
4,4,"美 의결권 자문사 글래스 루이스, 현대차그룹 지배구조 개편 반대의견",0
...,...,...,...
196,196,"코스피, 삼성電 '쇼크' 딛고 2거래일째 상승 마감",1
197,197,"코스피, 외인 5000억원 팔자에 2100선 '무너져'",-1
198,198,"코스피, 국제유가 상승VS 6월 변동 경계감…보합",0
199,199,코스피 2500 재진입 성공…외국인·기관 동반 매수,1


In [4]:
df_hb = pd.read_csv('./trainingset/현대백화점_traindata_200.csv')
df_hb

,id,Title,label
0,0,"靑 오찬도 없이 간담회 진행…재계 ""외교로 풀 문제를…""",0
1,1,"김정훈 ""공정위 소비자중심경영 인증기업 24%는 행정조치 처분기업""",-1
2,2,"산업부 ""코리아 세일 페스타, 상생의 장터 연다""",1
3,3,"박원순, 선거 마지막 날까지 정순균 강남구청장 후보 지원",0
4,4,"장영철 강남구청장 후보, 선거운동 마지막 주말 총력유세",0
...,...,...,...
216,216,입소문 노리려 인플루언서와 공간 나누는 유통업계(종합),1
217,217,"""인플루언서 모셔라""…'안방' 내주는 유통업계",1
218,218,"현대百, 고메 아이리시 위크…50여개 아일랜드 식품 브랜드 한자리에",1
219,219,풍성하고 즐거운 소비 '경험'…진열방식 바꾸는 패션가(종합),1


In [5]:
df_S_Oil = pd.read_csv('./trainingset/S-Oil_traindata_200.csv')
df_S_Oil = df_S_Oil.head(200)
# df_S_Oil.head(200)

In [6]:
df_IBK = pd.read_csv('./trainingset/기업은행_traindata_200.csv')
df_IBK

,id,Title,label
0,212,은행장 임기 줄줄이 만료…누가 연임될까,-1
1,1216,"권선주 기업은행장, 수능 앞둔 수험생에 격려편지",0
2,1461,서민 전세금 품어주는 HUG,1
3,788,"기업은행, 소방관 복지증진 위한 심신안정실 설치",1
4,1815,[2015 국감]3년간 은행·카드사 신분증 사본 5억여건 수집,0
...,...,...,...
195,769,"기업은행, 영업점장 대출 금리 감면권 확대",0
196,1827,"모임통장, 회비관리 '척척'…총무가 따로없네",0
197,793,"유니온페이, 겨울 여행객 위한 할인 이벤트 진행",0
198,68,"[2015 국감]이운룡 ""신보 채용사업, 구조조정 검토 필요""",-1


In [7]:
df_celt = pd.read_csv('./trainingset/셀트리온_traindata_200.csv')
df_celt=df_celt.head(200)
#df_celt

In [8]:
df_KAL = pd.read_csv('./trainingset/대한항공_testdata_300.csv', header=None)
df_celt=df_celt.head(200)
# df_KAL=df_KAL.head(300)
df_KAL=df_KAL.head(299)
df_KAL.columns = ['id','Title','label']
df_KAL

,id,Title,label
0,18,"""87 대선전 김현희 데려와야""",0.0
1,149,[반기문 방한]G7 참석차 일본행...27일 밤 서울로,0.0
2,167,해운 구조조정 기준 마련…조선·철강 머지않았다,-1.0
3,206,대한항공 호텔 철회 등 중재안 부상 경제활성화4法…통과 전망은,-1.0
4,125,北 미사일버튼 누르면 ‘실시간감지 정찰기’ 곧 전력화,1.0
...,...,...,...
294,294,"롯데홈쇼핑, 23일 '보물섬데이' 열어 여행상품 연속방송",1.0
295,295,[금융도우미]신한카드 대한항공 1000원 당 2마일리지 적립,1.0
296,296,"신한카드, 대한항공 마일리지 특화 ‘에어원’ 출시",1.0
297,297,"[일문일답]공정위 ""현대차 총수 정몽구 지정 전 건강소견서 받아""",0.0


In [9]:
df = pd.concat([df_train_skt, df_mobis, df_hb, df_S_Oil,df_IBK,df_celt])

In [10]:
df = df[['id','Title','label']]
df

,id,Title,label
0,0.0,국민안전로봇 2021년까지 개발…'로봇물고기' 전철 피할까?,0
1,1.0,'국정원 해킹 의혹' 상임위 앞두고 與野 전초전,0
2,2.0,인터넷티비(TV)결합상품 인터넷가입 비교사이트 통해 통신사별 장단점 비교해야 유리,0
3,3.0,SKT LG KT 인터넷가입 비교사이트 ‘펭귄통신’ 설치 당일 현금지원 이벤트,0
4,4.0,설 연휴 고향 가는 길 언제 출발해야 덜 막힐까,0
...,...,...,...
195,1172.0,"원 이퀴티 파트너스 포, 엘.피., 셀트리온헬스케어 주식 631만주 장내매도",-1
196,1630.0,"코스닥, 장 초반 하락세…640선 또 '위태'",-1
197,696.0,“ 3000% ”터질 바이오 단돈 3천원대! 2019년 마지막 新대장 바이오 온다!,1
198,1563.0,"코스닥, 개인 '사자'에 700 향해 ↑",1


In [11]:
from sklearn.model_selection import train_test_split

## 데이터셋 구성
- Train Dataset : skt 200개, 모비스 200개, 현백 200개, 에쓰오일 200개, 기업은행 200개, 셀트리온 200개
- Test Dataset : skt 282개

In [12]:
# Train 데이터와 Test 데이터 분리
X_train = df['Title']
y_train = df['label']
X_test = df_test_skt['Title']
y_test = df_test_skt['label']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 2)

In [13]:
# 문장을 나누기 위한 Stopwords 선정
stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

In [14]:
#!pip3 install konlpy

In [15]:
# Sentence Tokenizing

import konlpy
from konlpy.tag import Okt

okt = Okt()
X_train_token = []
for sentence in X_train:
    temp_X = [] 
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train_token.append(temp_X)

X_test_token = []
for sentence in X_test:
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test_token.append(temp_X)

In [16]:
from keras.layers import Embedding, Dense, LSTM
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences

### 단어길이 추출

In [17]:
li_train = []
# li_train.append(1)
# li_train.append(2)
for i in range(len(X_train_token)):
   li_train.append(len(X_train_token[i]))
max(li_train)

li_test = []
# li_train.append(1)
# li_train.append(2)
for i in range(len(X_test_token)):
   li_test.append(len(X_test_token[i]))
max(li_test)


66

### max_words를 35000에서 100으로 줄임(test 길이 중 가장 큰것은 66)

In [18]:
from keras.preprocessing.text import Tokenizer
max_words = 100
tokenizer = Tokenizer(num_words = max_words)
tokenizer.fit_on_texts(X_train_token) 
X_train_token = tokenizer.texts_to_sequences(X_train_token) 
X_test_token = tokenizer.texts_to_sequences(X_test_token)

### One-Hot encoding 2가지 방법 - 2번째 방법 사용 (to_categorize)

In [19]:
# 1. 
import numpy as np
y_train_encode = []
y_test_encode = []
for i in range(len(y_train)):
    if y_train.iloc[i] == 1:
        y_train_encode.append([0, 0, 1]) 
    elif y_train.iloc[i] == 0:
        y_train_encode.append([0, 1, 0]) 
    elif y_train.iloc[i] == -1:
        y_train_encode.append([1, 0, 0]) 
        
for i in range(len(y_test)):
    if y_test.iloc[i] == 1:
        y_test_encode.append([0, 0, 1]) 
    elif y_test.iloc[i] == 0:
        y_test_encode.append([0, 1, 0])
    elif y_test.iloc[i] == -1:
        y_test_encode.append([1, 0, 0]) 
        
y_train_encode=np.array(y_train_encode)
y_test_encode=np.array(y_test_encode)

In [20]:
len(y_train_encode)

1022

In [21]:
# 2. 
# to_categorical 방법은 0 1 -1 순서로 인코딩된다!
from keras.utils import to_categorical

y_train_cata = to_categorical(y_train, num_classes = 3)
y_test_cata = to_categorical(y_test, num_classes = 3)

In [22]:
y_train_cata

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

### max_len 20 to 100으로 변경 ; 단어 길이 고려

In [23]:
max_len = 100 # pad_sequesces이용, 전체 데이터의 길이를 20로 맞춘다
X_train = pad_sequences(X_train_token, maxlen=max_len)
X_test = pad_sequences(X_test_token, maxlen=max_len)

### LSTM 모델

optimizer = adam, epoch=10일 때, 테스트 결과 54.80%의 정확도를 보인다. 

### earlystopping과 modelcheckpoint 추가
### embedding 64로 변경

In [37]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=4)
mc = ModelCheckpoint('best_model.h5',monitor='val_acc',mode='max',verbose=1,save_best_only=True)

model = Sequential()
model.add(Embedding(max_words, 64))
model.add(LSTM(128))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train_cata, epochs=20, batch_size=10, callbacks=[es,mc], validation_split=0.1)

Epoch 1/20
110/110 [==============================] - 5s 47ms/step - loss: 1.0239 - accuracy: 0.5305 - val_loss: 0.9574 - val_accuracy: 0.5528
Epoch 2/20
110/110 [==============================] - 5s 46ms/step - loss: 0.9108 - accuracy: 0.5732 - val_loss: 0.8049 - val_accuracy: 0.6748
Epoch 3/20
110/110 [==============================] - 5s 46ms/step - loss: 0.8508 - accuracy: 0.6033 - val_loss: 0.7478 - val_accuracy: 0.7154
Epoch 4/20
110/110 [==============================] - 5s 49ms/step - loss: 0.8255 - accuracy: 0.6069 - val_loss: 0.7480 - val_accuracy: 0.6748
Epoch 5/20
110/110 [==============================] - 5s 49ms/step - loss: 0.8072 - accuracy: 0.6224 - val_loss: 0.7376 - val_accuracy: 0.6748
Epoch 6/20
110/110 [==============================] - 5s 46ms/step - loss: 0.7899 - accuracy: 0.6324 - val_loss: 0.7620 - val_accuracy: 0.6829
Epoch 7/20
110/110 [==============================] - 5s 48ms/step - loss: 0.7762 - accuracy: 0.6397 - val_loss: 0.8132 - val_accuracy: 0.6667

In [38]:
print("\n 테스트 정확도 : {:.2f}%".format(model.evaluate(X_test,y_test_cata)[1]*100))

9/9 [==============================] - 0s 17ms/step - loss: 0.9439 - accuracy: 0.6050

 테스트 정확도 : 60.50%


In [28]:
#optimizer = adam
model2 = Sequential()
model2.add(Embedding(max_words, 64))
model2.add(LSTM(128))
model2.add(Dense(3, activation='softmax'))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model2.fit(X_train, y_train_cata, epochs=10, batch_size=10, callbacks=[es,mc], validation_split=0.1)

Epoch 1/10
110/110 [==============================] - 6s 55ms/step - loss: 0.9997 - accuracy: 0.5205 - val_loss: 0.9637 - val_accuracy: 0.6098
Epoch 2/10
110/110 [==============================] - 6s 50ms/step - loss: 0.8888 - accuracy: 0.5769 - val_loss: 0.7858 - val_accuracy: 0.6829
Epoch 3/10
110/110 [==============================] - 6s 50ms/step - loss: 0.8283 - accuracy: 0.6087 - val_loss: 0.7372 - val_accuracy: 0.7073
Epoch 4/10
110/110 [==============================] - 6s 50ms/step - loss: 0.8064 - accuracy: 0.6133 - val_loss: 0.8370 - val_accuracy: 0.6829
Epoch 5/10
110/110 [==============================] - 5s 49ms/step - loss: 0.7841 - accuracy: 0.6315 - val_loss: 0.7923 - val_accuracy: 0.6829
Epoch 6/10
110/110 [==============================] - 6s 50ms/step - loss: 0.7736 - accuracy: 0.6306 - val_loss: 0.8076 - val_accuracy: 0.6667
Epoch 7/10
110/110 [==============================] - 6s 50ms/step - loss: 0.7612 - accuracy: 0.6297 - val_loss: 0.8642 - val_accuracy: 0.6829

In [29]:
print("\n 테스트 정확도 : {:.2f}%".format(model2.evaluate(X_test,y_test_cata)[1]*100))

9/9 [==============================] - 0s 16ms/step - loss: 0.9583 - accuracy: 0.6014

 테스트 정확도 : 60.14%


In [31]:
#model3 ; activation = relu
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=4)
mc = ModelCheckpoint('best_model.h5',monitor='val_acc',mode='max',verbose=1,save_best_only=True)

model3 = Sequential()
model3.add(Embedding(max_words, 64))
model3.add(LSTM(128))
model3.add(Dense(3, activation='relu'))
model3.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
history = model3.fit(X_train, y_train_cata, epochs=10, batch_size=10, callbacks=[es,mc], validation_split=0.1)

print("\n 테스트 정확도 : {:.2f}%".format(model3.evaluate(X_test,y_test_cata)[1]*100))

Epoch 1/10
110/110 [==============================] - 6s 57ms/step - loss: nan - accuracy: 0.2921 - val_loss: nan - val_accuracy: 0.1220
Epoch 2/10
110/110 [==============================] - 6s 51ms/step - loss: nan - accuracy: 0.2912 - val_loss: nan - val_accuracy: 0.1220
Epoch 3/10
110/110 [==============================] - 5s 50ms/step - loss: nan - accuracy: 0.2912 - val_loss: nan - val_accuracy: 0.1220
Epoch 4/10
110/110 [==============================] - 5s 49ms/step - loss: nan - accuracy: 0.2912 - val_loss: nan - val_accuracy: 0.1220
Epoch 00004: early stopping
9/9 [==============================] - 0s 16ms/step - loss: nan - accuracy: 0.5623

 테스트 정확도 : 56.23%


In [39]:
#model5 ; activation = softmax optimzier = nadam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=4)
mc = ModelCheckpoint('best_model.h5',monitor='val_acc',mode='max',verbose=1,save_best_only=True)

model5 = Sequential()
model5.add(Embedding(max_words, 64))
model5.add(LSTM(128))
model5.add(Dense(3, activation='softmax'))
model5.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model5.fit(X_train, y_train_cata, epochs=20, batch_size=10, callbacks=[es,mc], validation_split=0.1)

print("\n 테스트 정확도 : {:.2f}%".format(model5.evaluate(X_test,y_test_cata)[1]*100))

Epoch 1/20
110/110 [==============================] - 6s 59ms/step - loss: 1.0090 - accuracy: 0.5259 - val_loss: 1.0181 - val_accuracy: 0.5366
Epoch 2/20
110/110 [==============================] - 6s 54ms/step - loss: 0.9187 - accuracy: 0.5651 - val_loss: 0.8054 - val_accuracy: 0.6829
Epoch 3/20
110/110 [==============================] - 6s 54ms/step - loss: 0.8682 - accuracy: 0.5942 - val_loss: 0.7543 - val_accuracy: 0.6829
Epoch 4/20
110/110 [==============================] - 6s 54ms/step - loss: 0.8123 - accuracy: 0.6169 - val_loss: 0.7097 - val_accuracy: 0.7236
Epoch 5/20
110/110 [==============================] - 6s 54ms/step - loss: 0.8010 - accuracy: 0.6187 - val_loss: 0.7357 - val_accuracy: 0.7154
Epoch 6/20
110/110 [==============================] - 6s 54ms/step - loss: 0.7833 - accuracy: 0.6251 - val_loss: 0.7859 - val_accuracy: 0.7073
Epoch 7/20
110/110 [==============================] - 6s 54ms/step - loss: 0.7702 - accuracy: 0.6269 - val_loss: 0.7763 - val_accuracy: 0.6911